In [1]:
import pandas as pd
# import lightgbm as lgb
# import xgboost as xgb
# from catboost import CatBoostClassifier, Pool
from metrics import  accuracy
import numpy as np
import matplotlib.cm
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, SelectFromModel, RFE, VarianceThreshold, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from final_model_training import *
import lightgbm as lgbm
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB

# from catboost import CatBoostClassifier, Pool
import matplotlib.pyplot as plt

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning

simplefilter("ignore", category=ConvergenceWarning)
simplefilter("ignore", category=RuntimeWarning) 
simplefilter("ignore", category=UserWarning)

In [2]:
RANDOM_STATE = 42

x_train_path = "./../../data/x_train.txt"
y_train_path = "./../../data/y_train.txt"

x_test_path = "./../../data/x_test.txt"


X = pd.read_csv(x_train_path, delim_whitespace=True, header=None)
y = pd.read_csv(y_train_path, delim_whitespace=True, header=None)
X_real_test = pd.read_csv(x_test_path, delim_whitespace=True, header=None)


/tmp/ipykernel_15416/1583141569.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X = pd.read_csv(x_train_path, delim_whitespace=True, header=None)
/tmp/ipykernel_15416/1583141569.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(y_train_path, delim_whitespace=True, header=None)
/tmp/ipykernel_15416/1583141569.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_real_test = pd.read_csv(x_test_path, delim_whitespace=True, header=None)


# Ensembling #

In [3]:

xgb_params = {'colsample_bytree': 0.5307786809163018,
  'eta': 0.013200015577644199,
  'gamma': 3.3403122511768517,
  'max_depth': 9,
  'min_child_weight': 2,
  'n_estimators': 105,
  'reg_lambda': 0.2343443736214615,
  'seed': 123}

In [4]:
lgbm_params = {'colsample_bytree': 0.6489567476794972,
  'learning_rate': 0.25379898942316537,
  'min_child_weight': 10,
  'n_estimators': 145,
  'num_leaves': 2,
  'reg_lambda': 0.09233298220540166}

In [5]:
X_final = X[[100,102,105]]

In [6]:
cv = 5
reg_param = 0.4
model_grid_params = {
      "set1":[
            QuadraticDiscriminantAnalysis(reg_param=reg_param),
            XGBClassifier(**xgb_params),
            LGBMClassifier(**lgbm_params),
            GaussianNB(),
            SVC(kernel='linear', probability=True),
      ],
      "set2":[
            XGBClassifier(**xgb_params),
            LGBMClassifier(**lgbm_params),
      ],
      "set3":[
            QuadraticDiscriminantAnalysis(reg_param=reg_param),
            SVC(kernel='linear', probability=True),
            GaussianNB(),
      ],
      "set4":[
            QuadraticDiscriminantAnalysis(reg_param=reg_param),
      ]
}


In [7]:
results_dict = run_grid_search_cv(model_grid_params, X_final, y, profit_scoring, 5)

set1
Running on: set1
[LightGBM] [Info] Number of positive: 1997, number of negative: 2003
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000410 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 4000, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499250 -> initscore=-0.003000
[LightGBM] [Info] Start training from score -0.003000
[LightGBM] [Info] Number of positive: 1997, number of negative: 2003
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000075 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 4000, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499250 -> initscore=-0.003000
[LightGBM] [Info] Start training from score -0.003000


In [11]:
results_dict_mean = apply_transform_to_res(results_dict, np.std)
results_dict_mean

{'set1': 53.14132102234569,
 'set2': 47.49736834815167,
 'set3': 66.75327707311455,
 'set4': 64.18722614352485}

# Final #

In [10]:
X_final = X[[100,102,105]]
X_real_test_final = X_real_test[[100,102,105]]

model_set = [
        QuadraticDiscriminantAnalysis(reg_param=0.4),
        # XGBClassifier(**xgb_params),
        # LGBMClassifier(**lgbm_params),
        # GaussianNB(),
        # SVC(kernel='linear', probability=True),
]

train_and_predict_final_model(model_set, X_final, y, X_real_test_final)

,preds,id
4001,0.964595,4001
1109,0.891931,1109
2319,0.890909,2319
4997,0.888599,4997
3249,0.886925,3249
...,...,...
2838,0.561537,2838
3837,0.561523,3837
3784,0.561498,3784
2971,0.561470,2971
